# 🧠 Lecture 2: Controlling Access and Behavior in Objects
### Encapsulation • Properties • Magic Methods • Composition

## 1️⃣ Quick Review — OOP Part 1
- Classes are *blueprints* for creating objects.
- Objects store **state** (attributes) and **behavior** (methods).
- We used `__init__` to initialize state and `self` to refer to the object.
- OOP lets data and behavior live together — objects manage their own state.
- Procedural vs OOP: OOP bundles logic *with* data.

## 2️⃣ Direct Attribute Access — Why It’s a Problem
```python
class Car:
    def __init__(self, make, model, year, speed=0):
        self.make = make
        self.model = model
        self.year = year
        self.speed = speed

mycar = Car("Toyota", "Camry", 2022)
mycar.speed = -50   # ⚠️ logically invalid
```
Python allows this, but it violates **encapsulation** — we bypass the logic that should enforce valid values.

## 3️⃣ Python’s Data Model: "We’re All Consenting Adults"
Python trusts developers not to break things deliberately.  
Private attributes are only *conventionally* protected using a leading underscore (`_speed`).  
This means: you *can* access `_speed`, but you *shouldn’t*.  
> 🧭 Philosophy: *“We’re all consenting adults here.”*

## 4️⃣ Encapsulation with Getters and Setters (Traditional Way)
```python
class Car:
    def __init__(self, make, model, year, speed=0):
        self._make = make
        self._model = model
        self._year = year
        self._speed = speed

    def get_speed(self):
        return self._speed

    def set_speed(self, new_speed):
        if new_speed >= 0:
            self._speed = new_speed
        else:
            print("⚠️ Speed cannot be negative.")

mycar = Car("Toyota", "Camry", 2022)
mycar.set_speed(50)
print(mycar.get_speed())
```

## 5️⃣ The Pythonic Way — Using `@property`
We can make access look natural while keeping validation inside the class.
```python
class Car:
    def __init__(self, make, model, year, speed=0):
        self.make = make
        self._model = model
        self._speed = speed

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, value):
        if value >= 0:
            self._speed = value
        else:
            print("⚠️ Speed cannot be negative")

mycar = Car("Toyota", "Camry", 2022)
mycar.speed = 60  # calls setter
print(mycar.speed)  # calls getter
```

## 6️⃣ How `@property` Works Under the Hood
- The decorator turns your method into a **`property` object**, which implements `__get__` and `__set__`.
- When you access `obj.attr`, Python calls `Class.attr.__get__(obj)`.
- When you assign `obj.attr = value`, Python calls `Class.attr.__set__(obj, value)`.
- The value after `=` becomes the argument to your setter function.

So `mycar.speed = 60` → `Car.speed.__set__(mycar, 60)` → calls `fset(mycar, 60)`.

## 7️⃣ Magic Methods: `__str__` and `__add__`
### `__str__` controls how your object prints
```python
class Car:
    def __init__(self, make, model, year, speed=0):
        self.make = make
        self.model = model
        self.year = year
        self.speed = speed

    def __str__(self):
        return f"{self.year} {self.make} {self.model} traveling at {self.speed} mph"

print(Car("Tesla", "Model S", 2024, 70))
```

### `__add__` defines how `+` works
```python
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance

    def __add__(self, other):
        if isinstance(other, BankAccount):
            return BankAccount("Joint Account", self.balance + other.balance)
        raise TypeError("Can only add two BankAccount objects")

    def __str__(self):
        return f"{self.owner} — Balance: ${self.balance}"

acct1 = BankAccount("Alice", 500)
acct2 = BankAccount("Bob", 700)
joint = acct1 + acct2
print(joint)
```

## 8️⃣ How Python Knows Which `__add__` to Call
- `acct1 + acct2` → `acct1.__add__(acct2)`
- The *left operand’s* class defines what happens.
- Inside `__add__`, `self` is `acct1` and `other` is `acct2`.
- We can verify `type(other)` with `isinstance()`.
- If `__add__` isn’t defined, Python tries `acct2.__radd__(acct1)`.

## 9️⃣ Composition Example — Todo List Manager
We’ll build two classes:
- `Todo`: one task, with creation and due dates stored in a dictionary.
- `MyList`: manages a list of `Todo` objects.

In [ ]:
import datetime

class Todo:
    def __init__(self, text, created, due):
        try:
            created_date = datetime.date.fromisoformat(created)
            due_date = datetime.date.fromisoformat(due)
        except ValueError:
            raise ValueError("Dates must be in YYYY-MM-DD format (e.g., 2025-10-17)")

        self.data = {
            "text": text,
            "created": created_date,
            "due": due_date
        }

    def is_overdue(self):
        return self.data["due"] < datetime.date.today()

    def edit_text(self, new_text):
        self.data["text"] = new_text

    def __str__(self):
        status = "⚠️ OVERDUE" if self.is_overdue() else "✅ On time"
        return f"[{status}] {self.data['text']} (Created: {self.data['created']}, Due: {self.data['due']})"


class MyList:
    def __init__(self):
        self.todos = []

    def add(self, todo):
        if isinstance(todo, Todo):
            self.todos.append(todo)
        else:
            print("⚠️ Only Todo objects can be added.")

    def remove(self, index):
        if 0 <= index < len(self.todos):
            removed = self.todos.pop(index)
            print(f"🗑️ Removed: {removed.data['text']}")
        else:
            print("⚠️ Invalid index.")

    def show_all(self):
        if not self.todos:
            print("📝 No todos yet.")
        else:
            for i, todo in enumerate(self.todos, start=1):
                d = todo.data
                print(f"{i}. {d['text']} (Created: {d['created']}, Due: {d['due']})")

    def show_overdue(self):
        overdue = [t for t in self.todos if t.is_overdue()]

        if not overdue:
            print("🎉 No overdue tasks!")
        else:
            print("⚠️ Overdue tasks:")
            for t in overdue:
                print(f" - {t.data['text']} (Due: {t.data['due']})")

    
    def load_from_file(self, filename):
        """Load todos from a text file created by save_to_file()."""
        try:
            with open(filename, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split('|')
                if len(parts) != 3:
                    continue  # skip malformed lines

                text = parts[0].strip()
                created_str = parts[1].replace('Created:', '').strip()
                due_str = parts[2].replace('Due:', '').strip()

                try:
                    todo = Todo(text, created_str, due_str)
                    self.add(todo)
                except ValueError:
                    print(f"⚠️ Skipped invalid date format in line: {line.strip()}")

            print(f"📂 Loaded {len(self.todos)} todos from {filename}")

        except FileNotFoundError:
            print(f"❌ File not found: {filename}")




## 🔧 🔟 Exercises

### Exercise 1 — Custom `__str__`
Create a class `Book` with attributes `title`, `author`, `year`.  
Define `__str__` to print in the form:
```
📖 Title (Author, Year)
```

In [ ]:
# TODO: Create Book class with __str__ that formats output neatly
class Book:
    """
    Simple book record.

    Args:
        title (str): Book title.
        author (str): Author's name.
        year (int | str | None): Publication year (optional).
        pages (int | str | None): Page count (optional).
    """
    def __init__(self, title, author, year=None, pages=None):
        self.title = str(title).strip()
        self.author = str(author).strip()
        self.year = int(year) if year is not None and str(year).strip() != "" else None
        self.pages = int(pages) if pages is not None and str(pages).strip() != "" else None

    def __str__(self):
        # Example: “The Left Hand of Darkness” | by Ursula K. Le Guin | 1969 | 304 pp.
        parts = [f"“{self.title}”", f"by {self.author}"]
        if self.year is not None:
            parts.append(f"{self.year}")
        if self.pages is not None:
            parts.append(f"{self.pages} pp.")
        return " | ".join(parts)

### Exercise 2 — Implement `__add__`
Create a class `ShoppingCart` that holds a list of items.  
When you use `+`, combine two carts into one containing all items.

In [ ]:
# TODO: Implement ShoppingCart class with __add__ to merge carts
class ShoppingCart:
    """
    Holds item quantities in a dict: {name: qty}.
    Using + returns a NEW cart with summed quantities.
    """
    def __init__(self, cart):
        self.mycart = []
        for c in cart:
            self.myccart.append(c)

    def _add_(self, other):
        if not isinstance(other, ShoppingCart):
           print('you can only add shopping carts')
           return self 
        else:
         return self.mycart + other.mycart
    
daves = ShoppingCart([1,2,3]) 
petes = ShoppingCart([4,5,6]) 

print(daves + 5)
print(daves + petes)


NameError: name 'cart' is not defined

### Exercise 3 — Add Properties
Create a class `Rectangle` with private attributes `_width`, `_height`.  
Use `@property` to enforce positive values and a read-only `area`.

In [ ]:
# TODO: Create Rectangle class using @property for width, height, and area

### Exercise 4 — Extend Todo & MyList
Add methods:
- `edit_text()` to modify a todo’s description (already implemented above)
- `save_to_file(filename)` in `MyList` to write all todos to a text file

In [ ]:
# TODO: Test and extend the Todo and MyList classes with edit_text() and save_to_file()

## ✅ Summary
| Concept | Key Idea |
|----------|-----------|
| `@property` | Controls how attributes are read/written |
| `__str__` | Defines printable string form |
| `__add__` | Defines `+` behavior |
| Composition | One class holding instances of another |
| Dates | Convert strings with `datetime.date.fromisoformat()` |